### Import packages, connect to openEO, and define aoi

In [2]:
# import 
import openeo
import folium
import json
import shapely.geometry
from openeo.processes import ProcessBuilder
import os
import xarray as xr
import matplotlib.pyplot as plt


# connect to openeo
conn = openeo.connect("https://openeo.dataspace.copernicus.eu").authenticate_oidc()

# out dir
outdir = "./auxiliary_data/senales"

# aoi
aoi = json.load(open('auxiliary_data/senales/senales_wgs84.geojson'))

# define time period
time_period = ['2023-02-01', '2023-02-28']

# check the aoi
region = aoi['features'][0]['geometry']
geom = shapely.geometry.shape(region)
centroid = geom.centroid
center_latlon = [centroid.y, centroid.x]

m = folium.Map(location=center_latlon, zoom_start=10)

folium.GeoJson(aoi).add_to(m)
m

Authenticated using refresh token.


### Load data, create masks, define bands

In [4]:
# load collections from openeo
s2 = conn.load_collection(
    'SENTINEL2_L1C',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=["B02", "B03", "B04", "B08", "B11"])
 
s2_L2A = conn.load_collection(
    'SENTINEL2_L2A',
    spatial_extent=region,
    temporal_extent=time_period,
    bands=['SCL'])

worldcover = conn.load_collection(
    'ESA_WORLDCOVER_10M_2021_V2',
    spatial_extent=region,
    bands=['MAP']).resample_spatial(projection=32632)

dem = conn.load_collection(
    "COPERNICUS_30",
    spatial_extent=region,
    bands=["DEM"]).resample_spatial(projection=32632).reduce_dimension(dimension='t', reducer='mean') 
# t is reduced from dem because otherwise the dem would have several bands in t dimension. very strange

# choose water from worlcover and reduce t
water_mask = (worldcover == 80).reduce_dimension(dimension="t", reducer="mean")

# merge sentinel2s and water mask
s2 = s2.merge_cubes(s2_L2A).merge_cubes(water_mask)

# define bands
green = s2.band("B03")
swir = s2.band("B11")
nir = s2.band("B08")
scl = s2.band("SCL")
water = s2.band("MAP")

# NDSI
ndsi = (green - swir) / (green + swir)

# cloud mask
cloud_mask = ( (scl == 8) | (scl == 9) | (scl == 3) | (scl == 10) ) * 1.0 # times one forces to binary


### Combine masks to get snow map

In [5]:
valid_mask = (cloud_mask !=1) & (water !=1)

In [6]:
# valid_mask.download(os.path.join(outdir,'valid_mask.nc'))

In [7]:
snow_sure = (ndsi > 0.6) & (nir > 0.45) & valid_mask
no_snow_sure = (ndsi < 0) & valid_mask

In [6]:
# snow_sure.download(os.path.join(outdir,'snow_sure.nc'))
# no_snow_sure.download(os.path.join(outdir,'no_snow_sure.nc'))

In [7]:
# Combine to a snow_map: 0 = uncertain, 1 = sure no-snow, 2 = sure snow
snow_map = snow_sure.multiply(2) + no_snow_sure.multiply(1)

In [8]:
# snow_map.download(os.path.join(outdir,'snow_map.nc'))

### normalized distance

In [8]:
# add dummy bands dimension, because openEO backend wants it for some reason
snow_sure = snow_sure.add_dimension(name="bands", label="snow_sure", type="bands")

In [9]:
# valid_mask = valid_mask.add_dimension(name="bands", label="valid_mask", type="bands")

In [10]:
# snow_valid = snow_sure.merge_cubes(valid_mask)

In [11]:
# snow_valid.download(os.path.join(outdir,'snow_valid.nc'))

In [12]:
# print(snow_valid.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}} - ['snow_sure', 'valid_mask'] - ['t', 'x', 'y', 'bands'])


In [16]:
# apply distance udf on entire polygon
distance_udf = openeo.UDF.from_file("distance_udf.py")

norm_distance = snow_sure.apply_polygon(geometries=aoi, process=distance_udf)

In [17]:
# # download normalized distance
# norm_distance.download(os.path.join(outdir,'normalized_distance_snow.nc'))

### altitude mask

In [19]:
snow_sure_dem = snow_sure.merge_cubes(dem)

In [20]:
print(snow_sure_dem.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}} - ['snow_sure', 'DEM'] - ['t', 'x', 'y', 'bands'])


In [22]:
altitude_udf = openeo.UDF.from_file("altitude_mask_udf.py")

# apply altitude mask udf on entire polygon
altitude_mask = snow_sure_dem.apply_polygon(
    geometries=aoi,
    process=altitude_udf
)
# filter out the DEM that has no information
altitude_mask = altitude_mask.filter_bands(["snow_sure"])

In [23]:
# # download altitude mask
# altitude_mask.download(os.path.join(outdir,'altitude_mask.nc'))

In [24]:
# # submit job with a name
# job = altitude_mask.create_job(
#     title="altitude_mask"
# )

# job_id = job.job_id
# print(f"Job ID: {job_id}")

# # Start the job
# job.start_and_wait()

# # Download the result as NetCDF
# job.download_results(target=os.path.join(outdir))


### distance index

In [25]:
norm_distance = norm_distance.filter_bands(["snow_sure"])

In [26]:
print(norm_distance.metadata)
print(altitude_mask.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}} - ['snow_sure'] - ['t', 'x', 'y', 'bands'])
CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}} - ['snow_sure'] - ['t', 'x', 'y', 'bands'])


In [27]:
# combine normalized distance and altitude mask into distance index
distance_index = norm_distance * altitude_mask

In [28]:
# distance_valid = distance_index.merge_cubes(valid_mask)

In [29]:
# distance_index.download(os.path.join(outdir,'distance_index.nc'))

In [32]:
scale_udf = openeo.UDF.from_file("scale_distance_udf.py")

# scale the distance values to 0-255, 255=no data
scaled_distance_index = distance_index.apply_polygon(geometries=aoi, process=scale_udf)

scaled_distance_index = scaled_distance_index.filter_bands(["snow_sure"])

In [33]:
print(scaled_distance_index.metadata)

CollectionMetadata({'spatial': {'bbox': [[-180, -56, 180, 83]]}, 'temporal': {'interval': [['2015-07-04T00:00:00Z', None]]}} - ['snow_sure'] - ['t', 'x', 'y', 'bands'])


### Download result

In [35]:
# submit job with a name
job = scaled_distance_index.create_job(
    title="scaled_distance_index"
)

job_id = job.job_id
print(f"Job ID: {job_id}")

# Start the job
job.start_and_wait()

# Download the result as NetCDF
job.download_results(target=os.path.join(outdir))


Job ID: j-2508211256334a64abd68fe06634c401
0:00:00 Job 'j-2508211256334a64abd68fe06634c401': send 'start'
0:00:14 Job 'j-2508211256334a64abd68fe06634c401': created (progress 0%)
0:00:20 Job 'j-2508211256334a64abd68fe06634c401': created (progress 0%)
0:00:27 Job 'j-2508211256334a64abd68fe06634c401': created (progress 0%)
0:00:36 Job 'j-2508211256334a64abd68fe06634c401': created (progress 0%)
0:00:46 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:00:59 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:01:15 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:01:35 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:02:00 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:02:31 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:03:09 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:03:56 Job 'j-2508211256334a64abd68fe06634c401': running (progress N/A)
0:04:55 Job 'j-2508211

/tmp/ipykernel_651462/1953238887.py:13: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  job.download_results(target=os.path.join(outdir))
/home/jengblom/miniconda3/envs/snowmap/lib/python3.9/site-packages/openeo/rest/job.py:192: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
/home/jengblom/miniconda3/envs/snowmap/lib/python3.9/site-packages/openeo/rest/job.py:196: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


{PosixPath('auxiliary_data/senales/openEO_2023-02-02Z.tif'): {'bands': [{'name': 'snow_sure'}],
  'eo:bands': [{'name': 'snow_sure'}],
  'href': 'https://s3.waw3-1.openeo.v1.dataspace.copernicus.eu/openeo-data-prod-waw4-1/batch_jobs/j-2508211256334a64abd68fe06634c401/openEO_2023-02-02Z.tif?X-Proxy-Head-As-Get=true&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=e5d145b4c251418e8b0d9c27cbb3e53b%2F20250821%2Fwaw4-1%2Fs3%2Faws4_request&X-Amz-Date=20250821T130334Z&X-Amz-Expires=86400&X-Amz-SignedHeaders=host&X-Amz-Security-Token=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJyb2xlX2FybiI6ImFybjpvcGVuZW93czppYW06Ojpyb2xlL29wZW5lby1kYXRhLXByb2Qtd2F3NC0xLXdvcmtzcGFjZSIsImluaXRpYWxfaXNzdWVyIjoib3BlbmVvLnByb2Qud2F3My0xLm9wZW5lby1pbnQudjEuZGF0YXNwYWNlLmNvcGVybmljdXMuZXUiLCJodHRwczovL2F3cy5hbWF6b24uY29tL3RhZ3MiOnsicHJpbmNpcGFsX3RhZ3MiOnsiam9iX2lkIjpbImotMjUwODIxMTI1NjMzNGE2NGFiZDY4ZmUwNjYzNGM0MDEiXSwidXNlcl9pZCI6WyJkNzZhNDk2Yi02YjIzLTRkZjEtYjE3My04N2Y0ZjI0ZDFjMTEiXX0sInRyYW5zaXRpdmVfdGFnX2tleXMiOlsidX

In [34]:
# scaled_distance_index.download(os.path.join(outdir,'scaled_distance_index.nc'))

In [5]:
# import xarray as xr
# import matplotlib.pyplot as plt

# # save png
# sdi_ds = xr.open_dataset('./auxiliary_data/senales/scaled_distance_index.nc')

# plt.figure(figsize=(10, 8))
# sdi_ds['value'].isel(t=3).plot(cmap='gray', vmin=0, vmax=255)
# plt.title(f"Scaled distance index - 2023-02-05")
# plt.xlabel("x")
# plt.ylabel("y")
# plt.savefig('./auxiliary_data/senales/scaled_distance_index.png', bbox_inches='tight', pad_inches=0)
# plt.close()
